In [5]:
# Necessary imports
from langchain_community.tools import StructuredTool
from pydantic import BaseModel, Field
from langchain_groq import ChatGroq
from typing import Annotated
from langchain_core.tools import InjectedToolArg
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv
import requests
# Load api key from .env
load_dotenv()

# Model config
model = ChatGroq(model="llama-3.1-8b-instant")

# Input argument schema
class GetCurrencyConversionRate(BaseModel):
    base_currency: str = Field(required=True, description="Source currency for fetching conversion rate")
    target_currency: str = Field(required=True, description="Target currency for conversion rate")

# Tool to fetch conversion rate
def get_conversion_rate(base_currency: str, target_currency: str) -> float:
    """
    It is a function which fetches the current currency conversion rate, from base currency to target currency
    """
    url = f"https://v6.exchangerate-api.com/v6/18540b03aa1f010cf9395e87/pair/{base_currency}/{target_currency}"
    response = requests.get(url=url)
    return response.json()["conversion_rate"]

# Creating tool
conversion_rate_tool = StructuredTool.from_function(
    func=get_conversion_rate,
    args_schema=GetCurrencyConversionRate
)

# Input argument schema - for converting base currency into target currency
class Convert(BaseModel):
    base_currency_value: int = Field(required=True, description="base currency value which wants to get converted")
    conversion_rate_value: float = Field(required=True, description="current conversion rate value")

def convert(base_currency_value: int, conversion_rate_value: Annotated[float, InjectedToolArg]) -> float:
    """
    Convert base currency value into target currency value,
    by multiplying base currency value with conversion rate value.
    """
    return base_currency_value * conversion_rate_value

convert_tool = StructuredTool.from_function(
    func=convert,
    args_schema=Convert
)

# Tool binding
model = model.bind_tools(tools=[conversion_rate_tool, convert])

# Chat history
message_history = []

# User query
query = HumanMessage(content="""
What is the conversion factor between USD and INR ?
also can you convert 10 USD into INR .
""")
message_history.append(query)

# LLMs response to user's query
ai_message = model.invoke(query.content)
message_history.append(ai_message)

# Tool execution
# tool_result = 

/var/folders/2n/vnp09xnn0s35m3xr_5c7_b300000gn/T/ipykernel_56776/175863681.py:18: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  base_currency: str = Field(required=True, description="Source currency for fetching conversion rate")
/var/folders/2n/vnp09xnn0s35m3xr_5c7_b300000gn/T/ipykernel_56776/175863681.py:19: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  target_currency: str = Field(required=True, description="Target currency for conversion rate")
/var/folders/2n/vnp09xnn0s35m3xr_5c7_b300000gn/T/ip

In [11]:
ai_message.tool_calls

[{'name': 'get_conversion_rate',
  'args': {'base_currency': 'USD', 'target_currency': 'INR'},
  'id': 'pwfevfcb7',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency_value': 10, 'conversion_rate_value': '90.1096'},
  'id': '76nxdjzhx',
  'type': 'tool_call'}]

In [12]:
for tool_call in ai_message.tool_calls:
    # get conversion rate value - call 1st tool
    if tool_call["name"] == "get_conversion_rate":
        tool_message1 = conversion_rate_tool.invoke(tool_call)
        conversion_rate = conversion_rate_tool.invoke(tool_call).content
        # append into chat history
        message_history.append(tool_message1)
    
    # convert using conversion rate value - call 2nd tool
    if tool_call["name"] == "convert":
        tool_call["args"]["conversion_rate_value"] = conversion_rate
        tool_message2 = convert_tool.invoke(tool_call)
        message_history.append(tool_message2)

final_result = model.invoke(message_history).content
print(final_result)

The conversion factor between USD and INR is approximately 1 USD = 90.1096 INR.
The value of 10 USD in INR is approximately 906.096 INR.


In [13]:
ai_message.tool_calls

[{'name': 'get_conversion_rate',
  'args': {'base_currency': 'USD', 'target_currency': 'INR'},
  'id': 'pwfevfcb7',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency_value': 10, 'conversion_rate_value': '90.1096'},
  'id': '76nxdjzhx',
  'type': 'tool_call'}]

In [14]:
message_history

[HumanMessage(content='\nWhat is the conversion factor between USD and INR ?\nalso can you convert 10 USD into INR .\n', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'pwfevfcb7', 'function': {'arguments': '{"base_currency":"USD","target_currency":"INR"}', 'name': 'get_conversion_rate'}, 'type': 'function'}, {'id': '76nxdjzhx', 'function': {'arguments': '{"base_currency_value":10}', 'name': 'convert'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 369, 'total_tokens': 408, 'completion_time': 0.055044709, 'completion_tokens_details': None, 'prompt_time': 0.029630918, 'prompt_tokens_details': None, 'queue_time': 0.055840872, 'total_time': 0.084675627}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_1151d4f23c', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b881c-b224-7f93-bb1a-5c96de

In [15]:
conversion_rate

'90.1096'